# Imports

In [1]:
import pandas as pd
import numpy as np

import os
from tqdm import tqdm

from sympy import Symbol, solve # 방정식 해 구하기

from datetime import datetime

# Define Functions
* 필요 함수 정의

In [2]:
def preprocessing(network):
    """네트워크 객체의 컬럼 이름 및 순서를 정리해줌
    - O, D, FFT, alpha, Capacity, beta, Length 등 만 남기고 나머지는 지운다.
    """
    net = pd.DataFrame({
        'O' : network['Init node '],
        'D' : network['Term node '],
        'FFT' : network['Free Flow Time '],
        'Power' : network['Power'],
        'Capacity' : network['Capacity '],
        'B' : network['B'].round(2),
        'Length' : network['Length '],
        #'Toll' : network['Toll '],
        'Link_num' : [i for i in range(1, len(network)+1)],
        'x_n' : 0,
        'y_n' : 0,
        'Direction' : 0,
        'x_n1' : 0,
        'Cost' : 0
    })
   
    net = net.sort_values(by = ['O', 'D']) # net1 데이터프레임을 O, D에 대하여 오름차순 정렬
    net.set_index('Link_num', drop = False, inplace = True)# 링크 번호로 네트워크 각 링크의 인덱스를 지정
    
    return net

In [3]:
# net_cost.info()
# 데이터 축소 :: int64 -> int16, float64 -> float16으로 바꾸어 데이터 크기를 단축해 준다.

def type_change(network):
    """net_cost네트워크의 변수들 타입을 int16, float16으로 변화시켜 데이터 메모리 절감"""
    network[['O', 'D', 'FFT', 'Power', 'Length', 'Link_num']] = network[['O', 'D', 'FFT', 'Power', 'Length', 'Link_num']].astype(np.int16)
    network[['Capacity', 'B', 'x_n', 'y_n', 'Direction', 'x_n1', 'Cost']] = network[['Capacity', 'B', 'x_n', 'y_n', 'Direction', 'x_n1', 'Cost']].astype(np.float16)

    return network

In [4]:
def node_range(network):
    """Sioux-Falls Network의 각 기점(Origin)에서 출발하는 링크 번호의 범위를 데이터프레임에 저장하는 함수"""
    origin_nodes = network['O'].unique()
    start_list = []
    end_list = []
    
    for origin in origin_nodes:
        link_list = network[network['O'] == origin]['Link_num'] # 기점이 origin인 모든 링크 번호 리스트 추출
        link_list.sort_values() # 기점 origin에 연결된 링크 번호들을 오름차순으로 정렬
        
        start = link_list.iloc[0] # 링크 리스트의 첫번째 링크(가장 먼저 번호 링크)
        end = link_list.iloc[-1] # 링크 리스트의 마지막 링크(가장 마지막 번호 링크)
        
        start_list.append(start)
        end_list.append(end)
        
    node_link_df = pd.DataFrame({
        'origin' : origin_nodes,
        'start_link' : start_list,
        'end_link' : end_list
    })
    
    node_link_df.set_index('origin', drop = False, inplace = True)# 링크 번호로 네트워크 각 링크의 인덱스를 지정
    
    return node_link_df

In [5]:
def calculate_initial_cost(network):
    """
    BPR 함수를 사용하여 링크 통행비용을 계산하는 함수.
    최초 비용을 계산해주므로, Cost 자리에 Free Flow Time을 적용해야 한다.
    :: 필요 변수 :: FFT, Power, B, X0, Capacity, Length, Toll
    :: BPR 함수 :: Cost = Free Flow Time * (1 + (B * (X_n / Capacity))**Power)
    """
    network['Cost'] = network['FFT'] * (1 + network['B'] * ((network['x_n'] / network['Capacity'])**network['Power'])) #+ (net1['Length'] * net1['Toll'])
    
    return network

In [6]:
def calculate_iterative_cost(network):
    """
    BPR 함수를 사용하여 Iteration을 돌면서 링크 통행비용을 계산하는 함수. 
    매 iteration마다 갱신되는 x_n 등을 이용하므로, 매 iteration마다 다시 계산해주어야 한다.
    :: 필요 변수 :: FFT, Power, B, X0, Capacity, Length, Toll
    :: BPR 함수 :: Cost1 = Cost * (1 + (B * (X_n / Capacity))**Power)
    """
    network['Cost'] = network['Cost'] * (1 + network['B'] * ((network['x_n'] / network['Capacity'])**network['Power'])) #+ (net1['Length'] * net1['Toll'])
    
    return network

In [7]:
def tree_Dijkstra(origin, network):
    """Dijkstra 방법을 이용해 Shortest Path를 구하기 위한 경로 트리를 구해주는 과정.
    이때 network는 반드시 cost 컬럼을 포함하고 있어야 합니다."""
    
    SP_df = pd.DataFrame({
        'To' : network['O'].unique(),
        'From' : 0,
        'C1' : 999,
        'Fin' : 0
    })
    
    SP_df.loc[SP_df['To'] == origin, 'C1'] = 0 # 'To' 열 값이 origin과 같은 행의 C1 열 값을 0으로 치환하기
    SP_df = SP_df.sort_values(by = ['C1', 'To']).reset_index(drop = True) # cost_df 를 C1과 To값에 따라서 정렬하기
    
    node_link = node_range(network) # 각 origin을 기점으로 하는 링크번호 범위 데이터프레임
    
    while True:
        if sum(SP_df['Fin']) == len(SP_df):
            break

        a = SP_df.loc[0, 'To'] # cost_df의 1행 1열 데이터
        ac = SP_df.loc[0, 'C1'] # cost_df의 1행 3열 데이터
        
        SP_df.loc[0, 'Fin'] = 1
        SP_df = SP_df.sort_values(by = ['To']).reset_index(drop = True)
            
        # DN : Dijkstra next updating node, D : Dijkstra iteration term
        node_a = node_link[node_link['origin'] == a] # node_range의 링크 중, origin이 a인 행만 추출
        start_link_a = node_a['start_link'].iloc[0] # origin a에 연결된 link들 중 첫번째 번호
        end_link_a = node_a['end_link'].iloc[0] # origin a에 연결된 link들 중 마지막 번호
            
        DN = network[(network['Link_num'] >= start_link_a) & (network['Link_num'] <= end_link_a)]
        
        DN = DN[['D', 'Cost']]
        DN.rename(columns = {'D' : 'To', 'Cost' : 'C'}, inplace = True)
            
        SP_df = pd.merge(SP_df, DN, on = ['To'], how = 'left') 
            
        SP_df = SP_df.fillna(999) # SP_df의 C 값 중 값이 없어 NaN인 값들을 999로 채우기
        #SP_df = SP_df.sort_values(by = 'To').reset_index(drop = True)
            
        upset = list(SP_df[SP_df['C1'] > (SP_df['C'] + ac)]['To']) ######### 진짜 여기서 죽는줄알았음 헛갈림 주의
        
        #print(upset)
        #print(Pk)
            
        if len(upset) > 0:
            SP_df.loc[SP_df['To'].isin(upset), 'Fin'] = 0
            SP_df.loc[SP_df['To'].isin(upset), 'C1'] = SP_df.loc[SP_df['To'].isin(upset), 'C'] + ac
            SP_df.loc[SP_df['To'].isin(upset), 'From'] = a
            
        SP_df = SP_df.iloc[:, 0:-1]
        SP_df = SP_df.sort_values(by = ['Fin', 'C1', 'To']).reset_index(drop = True)
    
    SP_output = pd.DataFrame({
        'O' : SP_df['From'],
        'D' : SP_df['To'],
        'SP' : SP_df['C1']
    })
    
    SP_output = SP_output.sort_values(by = 'SP').reset_index(drop = True)
    
    return SP_output

In [8]:
def SPD_Dijkstra(origin, destination, network):
    """주어진 네트워크 및 비용(SP)을 바탕으로,
    origin >> destination으로 향하는 최단 경로를 구해주는 함수.
    network 데이터프레임은 반드시 cost 열을 포함하고 있어야 합니다."""
    
    mintree = tree_Dijkstra(origin, network)
    OD_SP = mintree[mintree['D'] == destination]
    
    while True:
        if OD_SP['O'].iloc[0] == origin:
            break
        
        added = mintree[mintree['D'] == OD_SP['O'].iloc[0]]
        OD_SP = pd.concat([added, OD_SP], ignore_index = True)
        
    # 각 O-D에 Node Link 붙이기 : network의 Link_num 이용
    OD_SP = pd.merge(OD_SP, network, on = ['O', 'D'], how = 'left')
    OD_SP = OD_SP[['O', 'D', 'SP', 'Link_num']]
    OD_SP.set_index('Link_num', drop = False, inplace = True)# 링크 번호로 네트워크 각 링크의 인덱스를 지정
        
    return OD_SP

In [9]:
def find_alpha(alpha_list):
    """Alpha 값들 중 0 이상 1 이하 실수값만을 반환"""
    for alpha in alpha_list:
        if alpha.is_real == True:
            if alpha <= 1 and alpha >= 0:
                #print(alpha)
                return alpha
            else:
                pass
        else:
            pass

In [10]:
def convergence_test(network):
    """네트워크에 대한 convergence test를 실행하는 함수.
    :: network :: 네트워크 데이터프레임. x_n, x_n1, Cost 열을 포함하고 있어야 함
    :: k :: 수렴 기준. 0.000001 정도로 잡아 본다."""
    sum_xn = sum(network['x_n']) # 분모
    sum_varx = sum((network['x_n1'] - network['x_n']) ** 2) # 분자
    
    criterion = sum_varx / sum_xn
        
    return criterion

# Data settings

In [11]:
data_dir = 'dataset'

In [12]:
network_file = 'Network.csv'
network_path = os.path.join(data_dir, network_file)

In [13]:
OD_file = 'Sioux_Falls_OD_mini.csv'
OD_path = os.path.join(data_dir, OD_file)

In [14]:
network = pd.read_csv(network_path, encoding = 'UTF-8')

In [15]:
OD = pd.read_csv(OD_path, encoding = 'UTF-8')

In [16]:
network

,Init node,Term node,Free Flow Time,Capacity,Length,B,Power,Speed.limit,Toll1,Toll2,Type
0,1,2,6,25900.200640,6,0.15,4,0,0,0,1
1,1,3,4,23403.473190,4,0.15,4,0,0,0,1
2,2,1,6,25900.200640,6,0.15,4,0,0,0,1
3,2,6,5,4958.180928,5,0.15,4,0,0,0,1
4,3,1,4,23403.473190,4,0.15,4,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
71,23,22,4,5000.000000,4,0.25,2,0,0,0,1
72,23,24,2,5078.508436,2,0.25,2,0,0,0,1
73,24,13,4,5091.256152,4,0.25,2,0,0,0,1
74,24,21,3,4885.357564,3,0.25,2,0,0,0,1


# Initialization
* **`N = 0`**
* 초기 비용 계산 : $x_0$ = 0 (모든 링크의 통행량 = 0)인 상태에서 비용 계산
* 초기 배정 : All-or-Nothing 방식으로 통행 배정, $x_1$ 획득

## Data Ready

In [17]:
net1 = preprocessing(network)

In [18]:
net1.head()

,O,D,FFT,Power,Capacity,B,Length,Link_num,x_n,y_n,Direction,x_n1,Cost
Link_num,,,,,,,,,,,,,
1,1,2,6,4,25900.200640,0.15,6,1,0,0,0,0,0
2,1,3,4,4,23403.473190,0.15,4,2,0,0,0,0,0
3,2,1,6,4,25900.200640,0.15,6,3,0,0,0,0,0
4,2,6,5,4,4958.180928,0.15,5,4,0,0,0,0,0
5,3,1,4,4,23403.473190,0.15,4,5,0,0,0,0,0


## 초기 비용 계산 : $t_a(x_a^0)$
* $x_0$ = 0 (전부 0)임을 계산하여 Cost 계산

In [19]:
net_cost = calculate_initial_cost(net1) # x0 

## $x_1$ 획득 : AON Assignment

In [20]:
# 각 OD별로 Dijkstra를 사용해 최단경로를 산출, 
# OD 테이블의 교통량(T)을 각 최단경로에 포함된 링크에 All-or-Nothing으로 배정하기

origin_list = OD['O'].unique()

for origin in tqdm(origin_list): # 각 Origin에 대하여

    each_origin_OD = OD[OD['O'] == origin]
    destination_list = each_origin_OD['D']
    
    for destination in destination_list: # 각 Origin의 Destin에 대하여
        # OD별 shortest path 산출, 링크 넘버 추출
        if origin != destination: # Cyclic flow 방지 : 기점과 종점이 같지 않은 경우만 다룬다
            SPD = SPD_Dijkstra(origin, destination, net_cost)
            link_list = list(SPD['Link_num'])
            # 각 링크넘버에 OD 테이블의 교통량(T) 더해주기
            add_volume = OD[(OD['O'] == origin) & (OD['D'] == destination)].iloc[0].loc['T'] # 각 통과 링크에 더해줄 OD 교통량 산정
            print(f'(O, D) = ({origin}, {destination}), Path Link_num list = {link_list}, Add_volume = {add_volume}')
            net_cost.loc[net_cost['Link_num'].isin(link_list), 'x_n'] += add_volume # All or Nothing 배정
        else:
            pass

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

(O, D) = (1, 2), Path Link_num list = [1], Add_volume = 200
(O, D) = (1, 12), Path Link_num list = [2, 7], Add_volume = 400


 25%|█████████████████████                                                               | 1/4 [00:00<00:01,  1.60it/s]

(O, D) = (1, 16), Path Link_num list = [1, 4, 16, 22], Add_volume = 1000
(O, D) = (2, 1), Path Link_num list = [3], Add_volume = 200


 50%|██████████████████████████████████████████                                          | 2/4 [00:01<00:01,  1.71it/s]

(O, D) = (2, 12), Path Link_num list = [3, 2, 7], Add_volume = 200
(O, D) = (2, 16), Path Link_num list = [4, 16, 22], Add_volume = 800
(O, D) = (12, 1), Path Link_num list = [35, 5], Add_volume = 400
(O, D) = (12, 2), Path Link_num list = [35, 5, 1], Add_volume = 200


 75%|███████████████████████████████████████████████████████████████                     | 3/4 [00:01<00:00,  1.77it/s]

(O, D) = (12, 16), Path Link_num list = [36, 32, 29], Add_volume = 1400
(O, D) = (16, 1), Path Link_num list = [47, 19, 14, 3], Add_volume = 1000


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.76it/s]

(O, D) = (16, 2), Path Link_num list = [47, 19, 14], Add_volume = 800
(O, D) = (16, 12), Path Link_num list = [48, 27, 33], Add_volume = 1400


In [21]:
net_cost

,O,D,FFT,Power,Capacity,B,Length,Link_num,x_n,y_n,Direction,x_n1,Cost
Link_num,,,,,,,,,,,,,
1,1,2,6,4,25900.200640,0.15,6,1,1400,0,0,0,6.0
2,1,3,4,4,23403.473190,0.15,4,2,600,0,0,0,4.0
3,2,1,6,4,25900.200640,0.15,6,3,1400,0,0,0,6.0
4,2,6,5,4,4958.180928,0.15,5,4,1800,0,0,0,5.0
5,3,1,4,4,23403.473190,0.15,4,5,600,0,0,0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,23,22,4,2,5000.000000,0.25,4,72,0,0,0,0,4.0
73,23,24,2,2,5078.508436,0.25,2,73,0,0,0,0,2.0
74,24,13,4,2,5091.256152,0.25,4,74,0,0,0,0,4.0


## Define Constant

In [22]:
## Sympy :: 방정식을 풀기 위한 패키지 이용. 방정식에 포함될 각 변수들을 미리 선언하기
d = Symbol('d') # direction
B = Symbol('B') # B. 0.15 있는 그거
t = Symbol('t') # t. Cost
c = Symbol('c') # capacity
x = Symbol('x') # x0
alpha = Symbol('alpha') # alpha
P = Symbol('P') # Power. 4.

In [23]:
k = 0.000001 ### Convergence test 통과 기준

# Iteration
* `while True:`를 사용해서 convergence test를 통과할 때까지 무한반복

In [24]:
print('==========================')
print(f'● Initialization')
print('==========================')
print(net_cost[['x_n', 'y_n', 'Cost', 'Direction', 'x_n1']])

start_time = datetime.now()

iteration = 1

while True:
    print('==========================')
    print(f'● Iteration : {iteration}')
    print('==========================')
    
    ######### Update Link Travel Time. t1계산. 통행비용을 cost 열에 갱신.
    ######### 전 iteration 단계에서 계산한 통행량 xn으로 통행비용 tn 계산함
    
    net_cost = calculate_iterative_cost(net_cost) 
    
    ######### 통행비용 tn(Cost)을 이용하여 Dijkstra 방식으로 Shortest Path를 산정
    ######### All-or-Nothing Assignment 실행, yn 획득
    
    for origin in tqdm(origin_list): # 각 Origin에 대하여

        each_origin_OD = OD[OD['O'] == origin]
        destination_list = each_origin_OD['D']
    
        for destination in destination_list: # 각 Origin의 Destin에 대하여
            # OD별 shortest path 산출, 링크 넘버 추출
            if origin != destination:
                SPD = SPD_Dijkstra(origin, destination, net_cost)
                link_list = list(SPD['Link_num'])
                # 각 링크넘버에 OD 테이블의 교통량(T) 더해주기
                add_volume = OD[(OD['O'] == origin) & (OD['D'] == destination)].iloc[0].loc['T'] # 각 통과 링크에 더해줄 OD 교통량 산정 
                net_cost.loc[net_cost['Link_num'].isin(link_list), 'y_n'] += add_volume # All or Nothing 배정 :: y0 갱신
            else:
                pass
        
    ######### Find Move Size Alpha. 목적함수 Z(x)를 최소화하는 Alpha 값 찾기
    ######### Find Direction d = yn - xn
    ######### sympy를 이용하여 0 이상 1 이하의 Alpha 실근을 구한다
    
    net_cost['Direction'] = net_cost['y_n'] - net_cost['x_n'] # direction :: d = yn - xn 임
    
    total_equation = 0 # total Equation : Z(x)가 최소화되기 위한 조건식

    for i in range(1, len(net_cost)+1): # 1부터 끝까지 Link_num == i에 대하여
    
        d = net_cost.loc[i, 'Direction'] # i링크의 direction
        t = net_cost.loc[i, 'Cost']
        B = net_cost.loc[i, 'B']
        x = net_cost.loc[i, 'x_n']
        P = net_cost.loc[i, 'Power']
        C = net_cost.loc[i, 'Capacity']
    
        equation = d * t * (1 + B * (((x + alpha * d)/C)**P))
        b
        total_equation += equation
    
    alpha_list = solve(total_equation)
    print(f'Alpha List : {alpha_list}')
    ith_alpha = find_alpha(alpha_list) # 여기서 alpha는 0과 1 사이의 양수여야 하므로, 해당 부분만을 필터링해주는 것이다.
    
    ######### x_n값 및 Alpha 이용, >> x_n1 컬럼 값 생성     
    net_cost['x_n1'] = net_cost['x_n'] + ith_alpha * (net_cost['Direction']) # 금번 iteration의 Alpha 값을 이용, x_n+1 값을 계산해준다.
    
    print(net_cost[['x_n', 'y_n', 'Cost', 'Direction', 'x_n1']].round(3))
    
    ######### Convergence Test 실행 :
    ######### 만약 Convergence Test를 통과할 경우 본 while문은 깨bb지고 결과를 출력하면 된다.
    ######### 만약 통과하지 못하면 본 iteration은 계속 반복된다.
    
    convergence = convergence_test(net_cost) # False가 저장될 시, 위 과정을 반복한다.
    print(f'convergence criterion = {convergence}')
    
    if convergence <= k : # Convergence Test 통과 시 While문이 종료된다.
        end_time = datetime.now()
        
        print(f'Process is finished : {start_time - end_time} seconds.')
        break
        
    else:
        net_cost['x_n'] = net_cost['x_n1']
        net_cost['y_n'] = 0 # 초기화해주지 않으면 엄청 늘어나버린다.
        net_cost['Direction'] = 0
        iteration +=1

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

● Initialization
           x_n  y_n  Cost  Direction  x_n1
Link_num                                  
1         1400    0   6.0          0     0
2          600    0   4.0          0     0
3         1400    0   6.0          0     0
4         1800    0   5.0          0     0
5          600    0   4.0          0     0
...        ...  ...   ...        ...   ...
72           0    0   4.0          0     0
73           0    0   2.0          0     0
74           0    0   4.0          0     0
75           0    0   3.0          0     0
76           0    0   2.0          0     0

[76 rows x 5 columns]
● Iteration : 1


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.94it/s]


Alpha List : []


TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

0

In [ ]:
sum(net_cost['y_n'])

In [ ]:
sum(net_cost['x_n1'])

# Save UE Result

In [ ]:
result_file_name = 'UE_result.csv'
result_file_path = os.path.join(data_dir, result_file_name)

In [ ]:
net_cost.to_csv(result_file_path, encoding = 'UTF-8')